# 네이버 블로그 크롤링 - 제주맛집

In [50]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote
import re

In [6]:
url = f'https://search.naver.com/search.naver?where=view&sm=tab_jum&query={quote("제주맛집")}'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
time.sleep(2)

In [12]:
from selenium.webdriver.common.keys import Keys

body=driver.find_element_by_tag_name('body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [1]:
soup=BeautifulSoup(driver.page_source)
lis = soup.select('.bx._svp_item')
len(lis)

NameError: name 'BeautifulSoup' is not defined

In [20]:
li = lis[0]
title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text().strip()
title

'만족뿐인 제주 맛집 공유'

In [22]:
date = li.select_one('.sub_time.sub_txt').get_text().strip()
date

'2021.11.19.'

In [23]:
desc = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
desc

'지난 주말 친구들과 제주여행을 떠났다가 이름알려진 제주 맛집에 갔다왔어요. 전복뚝배기부터 고등어조림,쌈밥 등 풍성한 음식에 아름다운 바다풍경까지 감상할 수 있었던 식당으로 만족스러웠던 곳이랍니다. 1....'

In [24]:
href = li.select_one('a.api_txt_lines.total_tit._cross_trigger')['href']
href

'https://blog.naver.com/tok4/222573156818'

- 날짜 처리

In [27]:
from datetime import datetime, timedelta

today = datetime.now()          #=datetime.today()
today

datetime.datetime(2021, 12, 8, 10, 45, 8, 219638)

In [28]:
today.year, today.month, today.day

(2021, 12, 8)

In [39]:
# 어제
date = today - timedelta(1)
f'{date.year}.{date.month:02d}.{date.day:02d}'

'2021.12.07'

In [40]:
# 6일전
date = today - timedelta(6)
f'{date.year}.{date.month:02d}.{date.day:02d}'


'2021.12.02'

In [38]:
#13시간전
date = today - timedelta(hours=13)
f'{date.year}.{date.month:02d}.{date.day:02d}'

'2021.12.07'

In [37]:
#45분전
date = today - timedelta(minutes=45)
f'{date.year}.{date.month:02d}.{date.day:02d}'

'2021.12.08'

In [147]:
def convert_date(s):
    #if len(s) ==11:
        #return s[:-1]           #로직 간단하게 강사님.
    today = datetime.now()
    if s=='어제':
        date = today - timedelta(1)       
    elif str(s[-3])=='일':              #'일' in s
        s = int(re.sub('[^0-9]','', s))     
        date = today - timedelta(s)       #date = today - timedelta(int(s[:-3]))
    elif str(s[-4])=='시':              #'시' in s
        s = int(re.sub('[^0-9]','', s))
        date = today - timedelta(hours=s)   #date = today - timedelta(int(s[:-4]))     
    elif str(s[-3])=='분':              #'분' in s
        s = int(re.sub('[^0-9]','', s))
        date = today - timedelta(minutes=s)     #date = today - timedelta(int(s[:-3]))
    else :
        return s     
    return f'{date.year}.{date.month:02d}.{date.day:02d}'
    



- 반복문으로 데이터 가져오기

In [149]:
lines = []
for li in lis:
    title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text().strip()
    date = li.select_one('.sub_time.sub_txt').get_text().strip()
    desc = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
    href = li.select_one('a.api_txt_lines.total_tit._cross_trigger')['href']

    lines.append([title,convert_date(date),desc,href])

In [150]:
df = pd.DataFrame(lines, columns=['제목','날짜','내용','href'])

In [152]:
df[['제목','날짜','내용']].head()

,제목,날짜,내용
0,만족뿐인 제주 맛집 공유,2021.11.19.,지난 주말 친구들과 제주여행을 떠났다가 이름알려진 제주 맛집에 갔다왔어요. 전복뚝배...
1,또 생각났던 제주 맛집,2021.12.02,제가 시킨 제주 비빔국수는 갖은 야채들이 그릇에 알록달록 예쁘게 가득 채워져 있었는...
2,북적였던 제주 맛집 리스트,2021.11.13.,무엇보다 비린내 한번 느껴지지 않아 배가 부른 상태였음에도 쉽사리 남길 수가 없었던...
3,감격했던 제주 맛집 소개,2021.11.15.,얼마 전 친구들과 제주로 여행갔다가 현지인께 추천받은 제주 맛집에 다녀왔어요. 갓 ...
4,제주도 가볼만한곳 제주 맛집,2021.12.01,제주도 가볼만한곳 제주 맛집은 깔끔한 메뉴가 푸짐하게 나오기로 소문 난 곳이라 부담...


In [153]:
df.to_csv('네이버-제주맛집.csv', index=False)

In [154]:
driver.close()